In [44]:
import numpy as np
import pandas as pd
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_columns',100)
import joblib

In [2]:
data = pd.read_excel(r"C:\Users\Pravin\Documents\Github\Bank Offer Recommdation System\Web-Scraping\consolidated_data.xlsx",engine='openpyxl')

In [3]:
data.head()

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Offer title (if available),Start date,End date,Detailed offer text (if available),Logo link,Offer image,Redeem link,Promo code (if available),Online/ Offline,"If offline, cities"
0,standard chartered,AMAZON SHOPPING VOUCHER,https://www.gyftr.com/sc-instavouchers/amazon-...,Retail,NaN,NaN,NaN,NaN,NaN,NaN,https://www.amazon.in/vouchers,NaN,Online,NaN
1,standard chartered,ARCHIES GALLERY,https://www.gyftr.com/sc-instavouchers/archies...,Gifts,NaN,NaN,NaN,Flat 10% OFF,NaN,NaN,https://www.archiesonline.com/,SCDISA0,Online,NaN
2,standard chartered,ARMANI EXCHANGE-LUXE GIFT CARD,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,NaN,NaN,NaN,Flat 10% OFF,NaN,NaN,NaN,SCDISAE0,Offline,NaN
3,standard chartered,ARROW,https://www.gyftr.com/sc-instavouchers/arrow-g...,Fashion,NaN,NaN,NaN,Flat 10% OFF,NaN,NaN,NaN,SCDISA0,Offline,NaN
4,standard chartered,AURELIA,https://www.gyftr.com/sc-instavouchers/aurelia...,Fashion,NaN,NaN,NaN,Flat 7.5% OFF,NaN,NaN,NaN,SCDIS0A0,Offline,NaN


In [4]:
data.drop(["If offline, cities","Start date","Logo link","Redeem link","Offer image","Offer title (if available)","End date"],axis=1,inplace=True)

In [5]:
data.head()

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
0,standard chartered,AMAZON SHOPPING VOUCHER,https://www.gyftr.com/sc-instavouchers/amazon-...,Retail,NaN,NaN,Online
1,standard chartered,ARCHIES GALLERY,https://www.gyftr.com/sc-instavouchers/archies...,Gifts,Flat 10% OFF,SCDISA0,Online
2,standard chartered,ARMANI EXCHANGE-LUXE GIFT CARD,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISAE0,Offline
3,standard chartered,ARROW,https://www.gyftr.com/sc-instavouchers/arrow-g...,Fashion,Flat 10% OFF,SCDISA0,Offline
4,standard chartered,AURELIA,https://www.gyftr.com/sc-instavouchers/aurelia...,Fashion,Flat 7.5% OFF,SCDIS0A0,Offline


In [8]:
data.isnull().sum()

Bank Name                                       0
Merchant Name                                   0
Link to offer on Bank site (for reference)      0
Category                                        0
Detailed offer text (if available)             41
Promo code (if available)                     213
Online/ Offline                                 0
dtype: int64

In [9]:
data.columns

Index(['Bank Name', 'Merchant Name',
       'Link to offer on Bank site (for reference)', 'Category',
       'Detailed offer text (if available)', 'Promo code (if available)',
       'Online/ Offline'],
      dtype='object')

## Univariant Analysis

In [10]:
data["Bank Name"].value_counts()

Bank Name
standard chartered      374
Indian Overseas Bank    149
PNB                      23
Name: count, dtype: int64

In [11]:
data.shape

(546, 7)

In [12]:
data.shape

(546, 7)

In [14]:
data["Merchant Name"] = data["Merchant Name"].str.title()

In [15]:
data[data["Merchant Name"].str.contains("Gift Card")]

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
2,standard chartered,Armani Exchange-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISAE0,Offline
12,standard chartered,Brooks Brothers -Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISOC10,Offline
19,standard chartered,Dune-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISMC10,Offline
30,standard chartered,G-Star-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISMC10,Offline
31,standard chartered,Gas-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISGAS0,Offline
35,standard chartered,Hamleys-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Toys & Games,Flat 10% OFF,SCDISHAM0,Offline
48,standard chartered,Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Gifts,Flat 10% OFF,SCDISOC10,Offline
51,standard chartered,Mothercare-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Kids,Flat 10% OFF,SCDISMC10,Offline
67,standard chartered,Replay- Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISMC10,Offline
74,standard chartered,Steve Madden-Luxe Gift Card,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,SCDISSM10,Offline


In [20]:

data.loc[data['Merchant Name'].str.contains('Gift Card', case=False), 'Promo code (if available)'] = 'No promocode'
# Remove 'Gift Card' text from 'merhcnay name'
data['Merchant Name'] = data['Merchant Name'].str.replace('Gift Card', '', case=False)
data['Merchant Name'] = data['Merchant Name'].str.replace('Shopping Voucher', '', case=False)

In [21]:
data

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
0,standard chartered,Amazon,https://www.gyftr.com/sc-instavouchers/amazon-...,Retail,NaN,NaN,Online
1,standard chartered,Archies Gallery,https://www.gyftr.com/sc-instavouchers/archies...,Gifts,Flat 10% OFF,SCDISA0,Online
2,standard chartered,Armani Exchange-Luxe,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,No promocode,Offline
3,standard chartered,Arrow,https://www.gyftr.com/sc-instavouchers/arrow-g...,Fashion,Flat 10% OFF,SCDISA0,Offline
4,standard chartered,Aurelia,https://www.gyftr.com/sc-instavouchers/aurelia...,Fashion,Flat 7.5% OFF,SCDIS0A0,Offline
...,...,...,...,...,...,...,...
541,Indian Overseas Bank,Bewakoof,https://www.iob.in/OfferZone.aspx,Fashion,15% off on a minimum transaction of Rs 499 at ...,NaN,Online
542,Indian Overseas Bank,Zee5,https://www.iob.in/OfferZone.aspx,Entertainment,FLAT 15% OFF on ZEE5 Premium HD and ZEE5 Premi...,NaN,Online
543,Indian Overseas Bank,Foxtale,https://www.iob.in/OfferZone.aspx,Entertainment,Flat 20% off on orders above 899,NaN,Online
544,Indian Overseas Bank,Kiro Beauty,https://www.iob.in/OfferZone.aspx,Personal Care,Flat 20% sitewide discount,NaN,Online


### Droping Duplicated Records

In [22]:
data[data[["Category","Detailed offer text (if available)"]].duplicated()]

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
3,standard chartered,Arrow,https://www.gyftr.com/sc-instavouchers/arrow-g...,Fashion,Flat 10% OFF,SCDISA0,Offline
7,standard chartered,Bewakoof,https://www.gyftr.com/sc-instavouchers/bewakoo...,Fashion,Flat 12.5% OFF,SCDISBK0,Online
8,standard chartered,Beyoung,https://www.gyftr.com/sc-instavouchers/beyoung...,Fashion,Flat 20% OFF,SCALL20,Online
12,standard chartered,Brooks Brothers -Luxe,https://www.gyftr.com/sc-instavouchers/luxe-gi...,Fashion,Flat 10% OFF,No promocode,Offline
13,standard chartered,Campus,https://www.gyftr.com/sc-instavouchers/campus-...,Fashion,Flat 10% OFF,SCDISMC10,Offline
...,...,...,...,...,...,...,...
512,Indian Overseas Bank,Colt,https://www.iob.in/OfferZone.aspx,Fashion,Buy for 1999/- and get extra 15% off,NaN,Online
514,Indian Overseas Bank,Anahi,https://www.iob.in/OfferZone.aspx,Fashion,Buy for 1999/- and get extra 15% off,NaN,Online
515,Indian Overseas Bank,Sugr,https://www.iob.in/OfferZone.aspx,Food,Buy for 1999/- and get extra 15% off,NaN,Online
527,Indian Overseas Bank,Pink Fort,https://www.iob.in/OfferZone.aspx,Fashion,Get 20% of on minimum purchase of Rs. 1999,NaN,Online


In [23]:
data.drop_duplicates(inplace=True)

In [24]:
data.shape

(355, 7)

### Missing Value Treatement

In [33]:
final_df = data.copy()

In [34]:
final_df.isnull().mean()

Bank Name                                     0.000000
Merchant Name                                 0.000000
Link to offer on Bank site (for reference)    0.000000
Category                                      0.000000
Detailed offer text (if available)            0.050704
Promo code (if available)                     0.535211
Online/ Offline                               0.000000
dtype: float64

In [35]:
final_df["Promo code (if available)"].fillna("No Promocode", inplace=True)

C:\Users\Pravin\AppData\Local\Temp\ipykernel_13608\2370099667.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df["Promo code (if available)"].fillna("No Promocode", inplace=True)


In [38]:
for index, row in final_df.iterrows():
    if pd.isnull(row['Detailed offer text (if available)']):
        final_df.at[index, 'Detailed offer text (if available)'] = f"brand voucher {row['Merchant Name'].lower()}"

In [39]:
final_df.isnull().sum()

Bank Name                                     0
Merchant Name                                 0
Link to offer on Bank site (for reference)    0
Category                                      0
Detailed offer text (if available)            0
Promo code (if available)                     0
Online/ Offline                               0
dtype: int64

### Model Building

In [40]:
final_df[["Merchant Name",'Category']].to_clipboard()

In [41]:
def build_recommendation_model(data):
    # Text data preprocessing and TF-IDF vectorization
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['Detailed offer text (if available)'].str.strip().str.replace("\n",""))

    # One-hot encode categorical variables
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(data[['Category',"Bank Name", "Merchant Name","Online/ Offline","Promo code (if available)"]])

    # Combine features
    combined_features = hstack([categorical_features])
    
    # normalized_features = normalize(combined_features)
    normalized_features = normalize(combined_features)
     # Calculate cosine similarity
    similarity_matrix = cosine_similarity(normalized_features, normalized_features)
    # Compute the cosine similarity between each movie description
    linear_similarity = linear_kernel(normalized_features, normalized_features)
    
    return similarity_matrix,linear_similarity

def recommend_similar_offers(offer_id, similarity_matrix, offers_df, top_n=10):
    similar_indices = similarity_matrix[offer_id].argsort()[-top_n-1:-1][::-1]
    similar_offers = offers_df.iloc[similar_indices]
    return similar_offers

# Build recommendation model using combined data
similarity_matrix,linear_similarity = build_recommendation_model(final_df)

# Get recommendations for a specific offer
offer_id = 200 #Change this to the offer ID you want recommendations for
recommended_offers = recommend_similar_offers(offer_id, similarity_matrix, final_df)
recommended_offers

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
391,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netmeds/,Healthcare,Get 3 Month Free NFM Membership,No Promocode,Online
388,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netme...,Healthcare,Get Rs. 200 off on all Tests and Packages,No Promocode,Online
452,Indian Overseas Bank,Netmeds,https://www.iob.in/OfferZone.aspx,Healthcare,Offer 1: 25% off on Prescription meds + 100% N...,No Promocode,Online
377,PNB,Thyrocare,https://offers.pnbcards.in/online-offers/thyro...,Healthcare,Get 25% off on Catalogue Rate,No Promocode,Online
382,PNB,Tata 1Mg,https://offers.pnbcards.in/online-offers/333/,Healthcare,1. Upto 25% off on Prescribed Medicines For Ne...,No Promocode,Online
393,PNB,Candy Floss,https://offers.pnbcards.in/online-offers/candy...,Food,"Get 20% off on Perfumes, Candles, Diffusers, C...",No Promocode,Online
385,PNB,Onefinerate.Com,https://offers.pnbcards.in/online-offers/onefi...,Travel,"Get 10% off on Total Bill, Maximum off up to R...",No Promocode,Online
457,Indian Overseas Bank,Redcliffe Labs,https://www.iob.in/OfferZone.aspx,Healthcare,FLAT 25% OFF on all health checkup packages ac...,No Promocode,Online
396,PNB,Myntra,https://offers.pnbcards.in/online-offers/myntr...,Fashion,Rs. 250 off on Minimum Purchase Value of Rs. 1499,No Promocode,Online
395,PNB,Fabhotels,https://offers.pnbcards.in/online-offers/fabho...,Hospitality,Get Rs. 750 off on Minimum Booking Value of Rs...,No Promocode,Online


In [42]:
def build_recommendation_model(data):
    # Text data preprocessing and TF-IDF vectorization
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['Detailed offer text (if available)'].str.strip().str.replace("\n",""))

    # One-hot encode categorical variables
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(data[['Category', "Merchant Name","Online/ Offline"]])

    # Combine features
    combined_features = hstack([categorical_features])
    
    # normalized_features = normalize(combined_features)
    normalized_features = normalize(combined_features)
     # Calculate cosine similarity
    similarity_matrix = cosine_similarity(normalized_features, normalized_features)
    # Compute the cosine similarity between each movie description
    linear_similarity = linear_kernel(normalized_features, normalized_features)
    
    return similarity_matrix,linear_similarity

def recommend_similar_offers(offer_id, similarity_matrix, offers_df, top_n=10):
    similar_indices = similarity_matrix[offer_id].argsort()[-top_n-1:-1][::-1]
    similar_offers = offers_df.iloc[similar_indices]
    return similar_offers

# Build recommendation model using combined data
similarity_matrix,linear_similarity = build_recommendation_model(final_df)

# Get recommendations for a specific offer
offer_id = 200 #Change this to the offer ID you want recommendations for
recommended_offers = recommend_similar_offers(offer_id, similarity_matrix, final_df)
recommended_offers

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
352,standard chartered,Netmeds,https://www.gyftr.com/sc-instavouchers/netmeds...,Healthcare,Flat 12.5% OFF,SCDISNM12,Online
452,Indian Overseas Bank,Netmeds,https://www.iob.in/OfferZone.aspx,Healthcare,Offer 1: 25% off on Prescription meds + 100% N...,No Promocode,Online
391,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netmeds/,Healthcare,Get 3 Month Free NFM Membership,No Promocode,Online
375,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netme...,Healthcare,Get Rs. 200 off on Medicine +80% NMS SuperCash,No Promocode,Online
457,Indian Overseas Bank,Redcliffe Labs,https://www.iob.in/OfferZone.aspx,Healthcare,FLAT 25% OFF on all health checkup packages ac...,No Promocode,Online
146,standard chartered,Healthians,https://www.gyftr.com/sc-instavouchers/healthi...,Healthcare,Flat 15% OFF,SCDISHEALTH0,Online
377,PNB,Thyrocare,https://offers.pnbcards.in/online-offers/thyro...,Healthcare,Get 25% off on Catalogue Rate,No Promocode,Online
382,PNB,Tata 1Mg,https://offers.pnbcards.in/online-offers/333/,Healthcare,1. Upto 25% off on Prescribed Medicines For Ne...,No Promocode,Online
112,standard chartered,Apollo Pharmacy,https://www.gyftr.com/sc-instavouchers/apollo-...,Healthcare,Flat 5% OFF,SCDISAP0,Online
164,standard chartered,Myupchar Medicines,https://www.gyftr.com/sc-instavouchers/myupcha...,Healthcare,Flat 1% OFF,SCDISM1,Online


In [49]:
def build_recommendation_model(data):
    # Text data preprocessing and TF-IDF vectorization
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['Detailed offer text (if available)'].str.strip().str.replace("\n",""))

    # One-hot encode categorical variables
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(data[['Category', "Merchant Name",]])

    # Combine features
    combined_features = hstack([categorical_features])
    
    # normalized_features = normalize(combined_features)
    normalized_features = normalize(combined_features)
     # Calculate cosine similarity
    similarity_matrix = cosine_similarity(normalized_features, normalized_features)
    # Compute the cosine similarity between each movie description
    linear_similarity = linear_kernel(normalized_features, normalized_features)
    
    return similarity_matrix,linear_similarity

def recommend_similar_offers(offer_id, similarity_matrix, offers_df, top_n=5):
    similar_indices = similarity_matrix[offer_id].argsort()[-top_n-1:-1][::-1]
    similar_offers = offers_df.iloc[similar_indices]
    return similar_offers

# Build recommendation model using combined data
similarity_matrix,linear_similarity = build_recommendation_model(final_df)

# Get recommendations for a specific offer
offer_id = 200 #Change this to the offer ID you want recommendations for
recommended_offers = recommend_similar_offers(offer_id, similarity_matrix, final_df)
recommended_offers

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
452,Indian Overseas Bank,Netmeds,https://www.iob.in/OfferZone.aspx,Healthcare,Offer 1: 25% off on Prescription meds + 100% N...,No Promocode,Online
352,standard chartered,Netmeds,https://www.gyftr.com/sc-instavouchers/netmeds...,Healthcare,Flat 12.5% OFF,SCDISNM12,Online
391,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netmeds/,Healthcare,Get 3 Month Free NFM Membership,No Promocode,Online
375,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netme...,Healthcare,Get Rs. 200 off on Medicine +80% NMS SuperCash,No Promocode,Online
520,Indian Overseas Bank,Medibuddy,https://www.iob.in/OfferZone.aspx,Healthcare,Full Body Health Checkup worth Rs.1899 at just...,No Promocode,Online


In [50]:


# Save the model
joblib.dump(similarity_matrix, 'recommendation_model_similarity_index.pkl')

['recommendation_model_similarity_index.pkl']

In [51]:
# Load the model
model = joblib.load('recommendation_model_similarity_index.pkl')

# Define a function to generate recommendations
def generate_recommendations(offer_id):
    # Get recommendations using the loaded model
    recommended_offers = recommend_similar_offers(offer_id, model, final_df)
    return recommended_offers

# Example usage:
offer_id = 200  # Change this to the offer ID you want recommendations for
recommended_offers = generate_recommendations(offer_id)
recommended_offers

,Bank Name,Merchant Name,Link to offer on Bank site (for reference),Category,Detailed offer text (if available),Promo code (if available),Online/ Offline
452,Indian Overseas Bank,Netmeds,https://www.iob.in/OfferZone.aspx,Healthcare,Offer 1: 25% off on Prescription meds + 100% N...,No Promocode,Online
352,standard chartered,Netmeds,https://www.gyftr.com/sc-instavouchers/netmeds...,Healthcare,Flat 12.5% OFF,SCDISNM12,Online
391,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netmeds/,Healthcare,Get 3 Month Free NFM Membership,No Promocode,Online
375,PNB,Netmeds,https://offers.pnbcards.in/online-offers/netme...,Healthcare,Get Rs. 200 off on Medicine +80% NMS SuperCash,No Promocode,Online
520,Indian Overseas Bank,Medibuddy,https://www.iob.in/OfferZone.aspx,Healthcare,Full Body Health Checkup worth Rs.1899 at just...,No Promocode,Online
